In [4]:
# %% % de familias con pérdida de empleo (Mar 2025 -> Jun 2025) - LÓGICA "AL MENOS UNO"
# Definición:
# - "Familia" = hogar con 1 o 2 representantes válidos (papá y/o mamá).
# - "Empleado" en un mes = aparece en Ingresos con tipo_empleo ∈ {Relación de Dependencia, Afiliación Voluntaria}.
# - "Elegible" = al menos 1 representante estaba empleado en marzo/2025.
# - "Pérdida"  = al menos 1 representante que estaba empleado en marzo/2025 deja de estarlo en junio/2025.
#   (Si ningún representante trabajaba en marzo, el hogar NO entra al denominador.)

import pandas as pd
from utils.carga_datos import cargar_datos_vulnerabilidad
from utils.hogarUnico import make_hogar_id

ANIO = 2025
MES_MARZO = 3
MES_JUNIO = 6
EMPLEOS_VALIDOS = {"Relacion de Dependencia", "Afiliacion Voluntaria"}


def _to_str_id(x):
    return str(x).strip() if pd.notna(x) else "0"


def _empleo_bool_map(df_ing: pd.DataFrame, anio: int, mes: int) -> dict:
    if df_ing.empty:
        return {}
    sub = df_ing[(df_ing["anio"] == anio) & (df_ing["mes"] == mes)].copy()
    if sub.empty:
        return {}
    sub["tipo_empleo"] = sub["tipo_empleo"].astype(str).str.strip()
    emp = (
        sub.assign(emp=sub["tipo_empleo"].isin(EMPLEOS_VALIDOS))
        .groupby("identificacion", as_index=False)["emp"]
        .max()
    )
    emp["identificacion"] = emp["identificacion"].astype(str).str.strip()
    return dict(zip(emp["identificacion"], emp["emp"]))


# --- Carga de datos
data = cargar_datos_vulnerabilidad()
df_personas = data.get("Personas", pd.DataFrame()).copy()
df_univ = data.get("Universo Familiares", pd.DataFrame()).copy()
df_ing = data.get("Ingresos", pd.DataFrame()).copy()

# Normalizar columnas clave
if not df_personas.empty:
    for c in ("periodo", "identificacion"):
        if c in df_personas.columns:
            df_personas[c] = df_personas[c].astype(str).str.strip()

if not df_univ.empty:
    for c in ("identificacion", "ced_padre", "ced_madre"):
        if c in df_univ.columns:
            df_univ[c] = df_univ[c].apply(_to_str_id).replace({"": "0", "nan": "0"})

if not df_ing.empty:
    for c in ("identificacion", "tipo_empleo"):
        if c in df_ing.columns:
            df_ing[c] = df_ing[c].astype(str).str.strip()

# Estudiantes (todos los periodos)
ids_est = (
    df_personas["identificacion"].dropna().unique().tolist()
    if not df_personas.empty
    else []
)

if not ids_est or df_univ.empty:
    print("% de familias con perdida de empleo en el ultimo trimestre: 0%")
else:
    # Universo acotado a esos estudiantes
    u = df_univ[df_univ["identificacion"].isin(ids_est)].copy()

    # Familias con al menos 1 representante válido
    u = u[(u["ced_padre"] != "0") | (u["ced_madre"] != "0")].copy()
    if u.empty:
        print("% de familias con perdida de empleo en el ultimo trimestre: 0%")
    else:
        # hogar_id único (independiente del orden)
        u["hogar_id"] = u.apply(
            lambda r: make_hogar_id(r["ced_padre"], r["ced_madre"]), axis=1
        )
        u = u[u["hogar_id"] != ""].copy()

        # Consolidar a un registro por hogar manteniendo reps (papá/mamá)
        u = u.drop_duplicates(subset=["hogar_id"])[
            ["hogar_id", "ced_padre", "ced_madre"]
        ].copy()

        # Mapas de empleo marzo y junio
        emp_mar = _empleo_bool_map(df_ing, ANIO, MES_MARZO)
        emp_jun = _empleo_bool_map(df_ing, ANIO, MES_JUNIO)

        elegibles = 0  # hogares con >=1 representante empleado en marzo
        perdidas = 0  # hogares donde AL MENOS UNO de los que trabajaban en marzo ya no trabaja en junio

        for _, r in u.iterrows():
            reps = []
            if r["ced_padre"] != "0":
                reps.append(str(r["ced_padre"]))
            if r["ced_madre"] != "0":
                reps.append(str(r["ced_madre"]))

            # ¿Quiénes trabajaban en marzo?
            trabajaban_mar = [rep for rep in reps if bool(emp_mar.get(rep, False))]
            if not trabajaban_mar:
                continue  # no elegible

            elegibles += 1

            # ¿Alguno de los que trabajaba en marzo dejó de trabajar en junio?
            alguno_perdio = any(
                not bool(emp_jun.get(rep, False)) for rep in trabajaban_mar
            )
            if alguno_perdio:
                perdidas += 1

        pct = round((perdidas / elegibles * 100.0), 2) if elegibles > 0 else 0.0
        pct_txt = f"{int(pct)}%" if pct.is_integer() else f"{pct}%"
        print(f"% de familias con perdida de empleo en el ultimo trimestre: {pct_txt}")

2025-09-23 16:35:45.699 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 16:35:45.700 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


% de familias con perdida de empleo en el ultimo trimestre: 3.54%


In [3]:
# %% Estudiantes cuyo hogar perdió al menos 1 empleo (Mar 2025 -> Jun 2025)
# Regla: Para el hogar del estudiante, si existe AL MENOS UN representante (papá o mamá)
#        que estaba empleado en marzo/2025 y NO está empleado en junio/2025 => +1 estudiante.
#
# Ejemplos:
# - Solo papá: trabajaba en marzo y no trabaja en junio => cuenta.
# - Papá y mamá: ambos trabajaban en marzo y en junio uno ya no => cuenta.
# - Nadie trabajaba en marzo => no cuenta (no hay “pérdida”).
#
# Salida por consola: solo líneas de texto por periodo y un total.

import pandas as pd
from utils.carga_datos import cargar_datos_vulnerabilidad

ANIO = 2025
MES_MAR = 3
MES_JUN = 6
EMPLEOS_VALIDOS = {"Relacion de Dependencia", "Afiliacion Voluntaria"}


def _to_str_id(x):
    return str(x).strip() if pd.notna(x) else "0"


def _empleo_bool_map(df_ing: pd.DataFrame, anio: int, mes: int) -> dict:
    """
    dict: identificacion (str) -> True/False si estuvo empleado ese mes.
    'Empleado' = tipo_empleo ∈ {RD, AV}. Si no aparece, False.
    """
    if df_ing.empty:
        return {}
    sub = df_ing[(df_ing["anio"] == anio) & (df_ing["mes"] == mes)].copy()
    if sub.empty:
        return {}
    sub["tipo_empleo"] = sub["tipo_empleo"].astype(str).str.strip()
    emp = (
        sub.assign(emp=sub["tipo_empleo"].isin(EMPLEOS_VALIDOS))
        .groupby("identificacion", as_index=False)["emp"]
        .max()
    )
    emp["identificacion"] = emp["identificacion"].astype(str).str.strip()
    return dict(zip(emp["identificacion"], emp["emp"]))


# --- Carga de datos
data = cargar_datos_vulnerabilidad()
df_personas = data.get("Personas", pd.DataFrame()).copy()
df_univ = data.get("Universo Familiares", pd.DataFrame()).copy()
df_ing = data.get("Ingresos", pd.DataFrame()).copy()

# Normalizaciones mínimas
if not df_personas.empty:
    for c in ("periodo", "identificacion"):
        if c in df_personas.columns:
            df_personas[c] = df_personas[c].astype(str).str.strip()

if not df_univ.empty:
    for c in ("identificacion", "ced_padre", "ced_madre"):
        if c in df_univ.columns:
            df_univ[c] = df_univ[c].apply(_to_str_id).replace({"": "0", "nan": "0"})

if not df_ing.empty:
    for c in ("identificacion", "tipo_empleo"):
        if c in df_ing.columns:
            df_ing[c] = df_ing[c].astype(str).str.strip()

# Mapas de empleo marzo y junio
emp_mar = _empleo_bool_map(df_ing, ANIO, MES_MAR)
emp_jun = _empleo_bool_map(df_ing, ANIO, MES_JUN)


def _estudiante_cuenta(est_id: str) -> bool:
    """
    True si en el hogar del estudiante hay AL MENOS un representante que
    trabajaba en marzo y dejó de trabajar en junio.
    """
    if df_univ.empty:
        return False

    u_rows = df_univ[df_univ["identificacion"] == est_id]
    if u_rows.empty:
        return False

    # Unir posibles filas (si hay varias) y reunir reps válidos
    reps = set()
    for _, r in u_rows.iterrows():
        p = _to_str_id(r.get("ced_padre", "0"))
        m = _to_str_id(r.get("ced_madre", "0"))
        if p != "0":
            reps.add(p)
        if m != "0":
            reps.add(m)

    if not reps:
        return False

    # ¿Algún representante trabajaba en marzo y dejó de trabajar en junio?
    for rep in reps:
        if bool(emp_mar.get(rep, False)) and (not bool(emp_jun.get(rep, False))):
            return True

    return False


# --- Conteo por periodo y total
if df_personas.empty:
    print("TOTAL: 0 estudiantes con pérdida de empleo en el hogar (Mar→Jun)")
else:
    periodos = sorted(df_personas["periodo"].dropna().unique().tolist())
    total = 0
    for per in periodos:
        ids_est = (
            df_personas.loc[df_personas["periodo"] == per, "identificacion"]
            .dropna()
            .astype(str)
            .str.strip()
            .drop_duplicates()
        )
        count_per = sum(_estudiante_cuenta(est_id) for est_id in ids_est)
        total += count_per
        print(
            f"Periodo {per}: {count_per} estudiantes con pérdida de empleo en el hogar (Mar→Jun)"
        )

    print(f"TOTAL: {total} estudiantes con pérdida de empleo en el hogar (Mar→Jun)")

2025-09-23 16:32:36.538 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 16:32:36.538 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Periodo 202520: 361 estudiantes con pérdida de empleo en el hogar (Mar→Jun)
TOTAL: 361 estudiantes con pérdida de empleo en el hogar (Mar→Jun)


In [1]:
# %% % de familias únicas con deudas de alto riesgo (criterio D/E y ratio >= 2.90)
# Definición:
# - "Familia" = hogar con 1 o 2 representantes válidos (papá y/o mamá).
# - Solo usa Ingresos: año 2025, mes 6 (salario mensual) y Deudas: año 2025, mes 7.
# - Deuda de alto riesgo (hogar):
#     * Existe deuda con cod_calificacion ∈ {D, E} en 2025-07
#     * ingreso_anual_hogar = (salario_junio_papa + salario_junio_mama) * 14
#     * ingreso_anual_hogar > 0
#     * (deuda_total_DE / ingreso_anual_hogar) >= 2.90
# - Denominador (hogares elegibles): hogares con ingreso_anual_hogar > 0
#   (porque si es 0 no se puede evaluar ratio)
#
# Salida: una sola línea con el porcentaje.

import pandas as pd
from utils.carga_datos import cargar_datos_vulnerabilidad
from utils.hogarUnico import make_hogar_id

ANIO_ING = 2025
MES_ING = 6
ANIO_DEU = 2025
MES_DEU = 7


def _to_str(x):
    return str(x).strip() if pd.notna(x) else "0"


# --- Cargar datos
data = cargar_datos_vulnerabilidad()
df_personas = data.get("Personas", pd.DataFrame()).copy()
df_univ = data.get("Universo Familiares", pd.DataFrame()).copy()
df_ing = data.get("Ingresos", pd.DataFrame()).copy()
df_deu = data.get("Deudas", pd.DataFrame()).copy()

# Normalizar claves
if not df_personas.empty:
    for c in ("periodo", "identificacion"):
        if c in df_personas.columns:
            df_personas[c] = df_personas[c].astype(str).str.strip()

if not df_univ.empty:
    for c in ("identificacion", "ced_padre", "ced_madre"):
        if c in df_univ.columns:
            df_univ[c] = df_univ[c].apply(_to_str).replace({"": "0", "nan": "0"})

if not df_ing.empty:
    for c in ("identificacion",):
        if c in df_ing.columns:
            df_ing[c] = df_ing[c].astype(str).str.strip()
    if "salario" in df_ing.columns:
        df_ing["salario"] = pd.to_numeric(df_ing["salario"], errors="coerce")

if not df_deu.empty:
    if "identificacion" in df_deu.columns:
        df_deu["identificacion"] = df_deu["identificacion"].astype(str).str.strip()
    if "valor" in df_deu.columns:
        df_deu["valor"] = pd.to_numeric(df_deu["valor"], errors="coerce")

# Subconjuntos requeridos
ing_jun = (
    df_ing[(df_ing["anio"] == ANIO_ING) & (df_ing["mes"] == MES_ING)].copy()
    if not df_ing.empty
    else pd.DataFrame()
)
deu_jul = (
    df_deu[(df_deu["anio"] == ANIO_DEU) & (df_deu["mes"] == MES_DEU)].copy()
    if not df_deu.empty
    else pd.DataFrame()
)

# Mapas ingreso mensual junio
sal_map = {}
if not ing_jun.empty and "salario" in ing_jun.columns:
    s_tmp = ing_jun.groupby("identificacion", as_index=False)["salario"].sum()
    sal_map = dict(zip(s_tmp["identificacion"], s_tmp["salario"]))

# Estudiantes y universo
ids_est = (
    df_personas["identificacion"].dropna().unique().tolist()
    if not df_personas.empty
    else []
)

if not ids_est or df_univ.empty:
    print("% de familias con deudas de alto riesgo (D/E, ratio>=2.90): 0%")
else:
    u = df_univ[df_univ["identificacion"].isin(ids_est)].copy()
    # Hogares con al menos 1 representante válido
    u = u[(u["ced_padre"] != "0") | (u["ced_madre"] != "0")].copy()
    if u.empty:
        print("% de familias con deudas de alto riesgo (D/E, ratio>=2.90): 0%")
    else:
        # hogar_id único
        u["hogar_id"] = u.apply(
            lambda r: make_hogar_id(r["ced_padre"], r["ced_madre"]), axis=1
        )
        u = u[u["hogar_id"] != ""].copy()
        # Un registro por hogar
        u = u.drop_duplicates(subset=["hogar_id"])[
            ["hogar_id", "ced_padre", "ced_madre"]
        ].copy()

        elegibles = 0  # hogares con ingreso anual > 0
        alto_riesgo = 0

        # Filtrar deudas D/E una sola vez
        de_de = pd.DataFrame()
        if not deu_jul.empty and "cod_calificacion" in deu_jul.columns:
            de_de = deu_jul[deu_jul["cod_calificacion"].isin(["D", "E"])].copy()

        for _, r in u.iterrows():
            reps = []
            if r["ced_padre"] != "0":
                reps.append(r["ced_padre"])
            if r["ced_madre"] != "0":
                reps.append(r["ced_madre"])

            # Ingreso anual (junio * 14) sumando reps
            ingreso_mensual = sum(float(sal_map.get(rep, 0.0)) for rep in reps)
            ingreso_anual = ingreso_mensual * 14.0

            if ingreso_anual <= 0:
                # No elegible para ratio
                continue

            elegibles += 1

            # Deuda total D/E del hogar
            deuda_total = 0.0
            if not de_de.empty and "valor" in de_de.columns:
                deuda_total = (
                    de_de[de_de["identificacion"].isin(reps)]["valor"].fillna(0).sum()
                )

            if deuda_total > 0 and (deuda_total / ingreso_anual) >= 2.90:
                alto_riesgo += 1

        pct = round((alto_riesgo / elegibles * 100.0), 2) if elegibles > 0 else 0.0
        pct_txt = f"{int(pct)}%" if pct.is_integer() else f"{pct}%"
        print(f"% de familias con deudas de alto riesgo (D/E, ratio>=2.90): {pct_txt}")

2025-09-23 16:41:47.376 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 16:41:47.376 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-09-23 16:42:10.812 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 16:42:10.812 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-23 16:42:10.813 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


% de familias con deudas de alto riesgo (D/E, ratio>=2.90): 1.71%
